In [38]:
import pandas as pd
from datetime import datetime

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

df = pd.read_csv("Tokopedia_PlayStore 1000 PlayStore indonesian reviews sentiment.csv", header=0, parse_dates=[0], index_col=0)

In [39]:
import torch
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
# Loading the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('cahya/bert-base-indonesian-522M')
tokenizer = BertTokenizer.from_pretrained('cahya/bert-base-indonesian-522M', do_lower_case=True)

In [41]:
summaries = {}
originals = []
sentiments = []
summary = []

In [42]:
df.head()

,content,sentiment
0,"Meresahkan. Iya sih promonya menarik, tapi kur...",-1
1,Di tingkatkan lagi informasi pelacakan barang ...,0
2,rewel di tagih terus kaya bocah .satu hari.lam...,-1
3,"Kurangin bintang lah,biaya aplikasi naik",-1
4,"suka sama paket belanja 10K dari tokped, semog...",1


In [10]:
# # Convert the index to a monthly period
# df.index = df.index.to_period('M')

# # Group the dataframe by month
# grouped_df = df.groupby(df.index)

# # Iterate over the groups and access the data for each month
# for month, data in grouped_df:
#     print(f"Month: {month}")


In [43]:
count = 1
for text,sentiment in zip(df['content'],df['sentiment']):
    # Preprocessing the text
    if isinstance(text, str):  # Check if text is a string
        sentences = sent_tokenize(text)
        if len(sentences) < 2:
            summary.append(text)
        else:
            tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

            # Encoding the input
            max_len = 0
            for i in tokenized_sentences:
                if len(i) > max_len:
                    max_len = len(i)

            padded_sentences = []
            for i in tokenized_sentences:
                while len(i) < max_len:
                    i.append(0)
                padded_sentences.append(i)

            input_ids = torch.tensor(padded_sentences)

            # Generating the sentence embeddings
            with torch.no_grad():
                last_hidden_states = model(input_ids)[0]

            sentence_embeddings = []
            for i in range(len(sentences)):
                sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

            # Summarizing the text
            # Compute the similarity matrix
            similarity_matrix = cosine_similarity(sentence_embeddings)

            # Generate the summary
            num_sentences = 1
            summary_sentences = []
            for i in range(num_sentences):
                sentence_scores = list(enumerate(similarity_matrix[i]))

            sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
            summary_sentences.append(sentences[sentence_scores[1][0]])

            summarization = ' '.join(summary_sentences)
            
            summary.append(summarization)
            
    else:
        summary.append(text)
    sentiments.append(sentiment)
    originals.append(text)

    if count % 100 == 0:
        print(f"{count} DONE")
    count += 1

100 DONE
200 DONE
300 DONE
400 DONE
500 DONE
600 DONE
700 DONE
800 DONE
900 DONE
1000 DONE


In [13]:
# for month, data in grouped_df:
#     text = ' '.join(data['review'].astype(str).tolist())
    
#     if len(text) > 10000:
#         text = text[:10000]

#     # Preprocessing the text

#     sentences = sent_tokenize(text)
#     tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

#     # Encoding the input
#     max_len = 0
#     for i in tokenized_sentences:
#         if len(i) > max_len:
#             max_len = len(i)

#     padded_sentences = []
#     for i in tokenized_sentences:
#         while len(i) < max_len:
#             i.append(0)
#         padded_sentences.append(i)

#     input_ids = torch.tensor(padded_sentences)

#     # Generating the sentence embeddings
#     with torch.no_grad():
#         last_hidden_states = model(input_ids)[0]

#     sentence_embeddings = []
#     for i in range(len(sentences)):
#         sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

#     # Summarizing the text
#     # Compute the similarity matrix
#     similarity_matrix = cosine_similarity(sentence_embeddings)

#     # Generate the summary
#     num_sentences = 1
#     summary_sentences = []
#     for i in range(num_sentences):
#         sentence_scores = list(enumerate(similarity_matrix[i]))

#     sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
#     summary_sentences.append(sentences[sentence_scores[1][0]])

#     summarization = ' '.join(summary_sentences)

#     summary.append(summarization)

#     originals.append(text)
#     print(f"{month} DONE")

In [14]:
# months = []
# for month, data in grouped_df:
#     months.append(month)
#     print(f"{month} DONE")

In [44]:
# summaries['Month'] = months
summaries['Original'] = originals
summaries['Summary'] = summary
summaries['Sentiment'] = sentiments

df_summaries = pd.DataFrame(summaries)

df_summaries.head()

,Original,Summary,Sentiment
0,"Meresahkan. Iya sih promonya menarik, tapi kur...","Iya sih promonya menarik, tapi kurirnya ga ber...",-1
1,Di tingkatkan lagi informasi pelacakan barang ...,Informasi saja sudah cukup.. di tingkatkan lag...,0
2,rewel di tagih terus kaya bocah .satu hari.lam...,rewel di tagih terus kaya bocah .satu hari.lam...,-1
3,"Kurangin bintang lah,biaya aplikasi naik","Kurangin bintang lah,biaya aplikasi naik",-1
4,"suka sama paket belanja 10K dari tokped, semog...","suka sama paket belanja 10K dari tokped, semog...",1


In [45]:
len(df_summaries)

1000

In [46]:
df_summaries.to_csv("Summarization 1000 sentiment tokopedia playstore.csv")